In [1]:
import sys
from timeit import default_timer as timer

sys.path.append('../pmm/')

import inputs
import pmm
import conversions
import MDAnalysis as mda
import numpy as np
import nglview as nv
from nglview.datafiles import GRO, XTC

In [2]:
# gather the electronic properties
start = timer()
pmm_inputs = inputs.get_tot_input_gauss('thioindigo/thio_pbe0_esp{}_x180.log', 16)
end = timer()
print('inputs da Gaussian ci mette: ', end - start)
#print(pmm_inputs)

qc_geom = pmm.convert2Universe(pmm_inputs['geometry'])
print(qc_geom)

inputs da Gaussian ci mette:  0.030039699995541014
[1.0, 12.0, 12.0, 1.0, 12.0, 1.0, 12.0, 12.0, 12.0, 1.0, 32.0, 12.0, 12.0, 16.0, 12.0, 12.0, 16.0, 12.0, 12.0, 1.0, 12.0, 1.0, 12.0, 1.0, 12.0, 1.0, 12.0, 32.0]
<Universe with 28 atoms>
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:467: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:473: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


In [ ]:
strat = timer()
traj = mda.Universe('thioindigo/thioindigo_trans_etoh_md.tpr', 'thioindigo/thio_etoh_center_10ps.xtc')
end = timer()